In [1]:
!pip install tabpfn --no-index --find-links=file:///kaggle/input/pip-packages-icr/pip-packages
!pip install ipywidgets
!mkdir -p /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff
!cp /kaggle/input/pip-packages-icr/pip-packages/prior_diff_real_checkpoint_n_0_epoch_100.cpkt /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff/

Looking in links: file:///kaggle/input/pip-packages-icr/pip-packages
mkdir: /opt/conda: Permission denied
cp: directory /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff does not exist


In [2]:
# Import Joblib Module from Scikit Learn
import joblib

import numpy as np                       # NumPy for numerical computations
import pandas as pd                      # Pandas for data manipulation and analysis
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, normalize   # LabelEncoder for encoding categorical variables, normalize for feature scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier   # GradientBoostingClassifier and RandomForestClassifier for classification models
from tabpfn import TabPFNClassifier 
import xgboost   # XGBoost for gradient boosting models
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score   # accuracy_score for evaluating model performance
from sklearn.impute import SimpleImputer   # SimpleImputer for handling missing values
import imblearn   # imblearn for imbalanced dataset handling
from imblearn.over_sampling import RandomOverSampler   # RandomOverSampler for oversampling minority class
from imblearn.under_sampling import RandomUnderSampler   # RandomUnderSampler for undersampling majority class
import inspect   # inspect for retrieving information about live objects
from collections import defaultdict   # defaultdict for creating a dictionary with default values
import warnings   # warnings for ignoring warnings during runtime
from sklearn.model_selection import KFold as KF
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import sys
import os

/Users/hadangvu/anaconda3/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Load Dataset

In [111]:
train = pd.read_csv('../datasets/icr-identify-age-related-conditions/train.csv')
greeks = pd.read_csv('../datasets/icr-identify-age-related-conditions/greeks.csv')
test = pd.read_csv('../datasets/icr-identify-age-related-conditions/test.csv')

In [112]:
#drop 'Id' 'EJ' columns
train = train.drop(['Id','EJ'], axis=1)

#fill 'median' for missing values
columns = train.columns
imputer = SimpleImputer(missing_values = np.nan, strategy ='median')
imputer = imputer.fit(train)
train = imputer.transform(train)
train = pd.DataFrame(train, columns = columns)

/Users/hadangvu/anaconda3/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/Users/hadangvu/anaconda3/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/Users/hadangvu/anaconda3/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated

## Cross Validation

In [113]:
NUM_FOLDS = 5
skf = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)
splitter = skf.split(train, train.Class)

for fold_idx, (train_idx, val_idx) in enumerate(splitter):
    print(f'Getting fold number {fold_idx}')
    x_train, y_train = train.iloc[train_idx], greeks.Alpha.iloc[train_idx]
    df_train = pd.concat((x_train, y_train), axis=1)
    df_val = train.iloc[val_idx]
    
    #drop column Id & reset index
    
#     df_train = df_train.drop(['Id'], axis=1)
    df_train = df_train.reset_index(drop = True)
#     df_val = df_val.drop(['Id'], axis=1)
    df_val = df_val.reset_index(drop = True)
    
    #kfold path
    save_dir = f'../datasets/kfold/fold{fold_idx}'
    os.makedirs(save_dir, exist_ok = True)
    
    #saving
    df_train.to_csv(os.path.join(save_dir, 'train.csv'), index = False)
    df_val.to_csv(os.path.join(save_dir, 'val.csv'), index = False)
    
    # for testing
    save_dir1 = f'../datasets/kfold1/fold{fold_idx}'
    os.makedirs(save_dir1, exist_ok = True)
    df_train = df_train.reset_index(drop=True)
    df_val = df_val.reset_index(drop=True)
    df_train.to_csv(os.path.join(save_dir1, 'train.csv'))
    df_val.to_csv(os.path.join(save_dir1, 'val.csv'))

Getting fold number 0
Getting fold number 1
Getting fold number 2
Getting fold number 3
Getting fold number 4


## Pre-processing

In [114]:
def prepair_test(df):
    #take 'Id' column and drop 'Id', 'EJ' columns
    Id = df['Id']
    test = df.drop(['Id', 'EJ'], axis=1)
    columns = test.columns
    
    imputer = SimpleImputer(missing_values = np.nan, strategy ='median')
    imputer = imputer.fit(test)
    test = imputer.transform(test)
    test = pd.DataFrame(test, columns = columns)
    return Id, test

def prepair_input(df, classi):
    columns = df.columns
    
    # Convert the values in the 'EJ' column of the 'test' dataframe to binary values (0 or 1),
    # based on the occurrence of the 'first_category' in the 'train' dataframe
#     first_category = df.EJ.unique()[0]
#     df.EJ = df.EJ.eq(first_category).astype('int')

    df = df.rename(columns={'BD ': 'BD', 'CD ': 'CD', 'CW ': 'CW', 'FD ': 'FD'})
    
    imputer = SimpleImputer(missing_values = np.nan, strategy ='median')
    imputer = imputer.fit(df)
    df = imputer.transform(df)
    df = pd.DataFrame(df, columns = columns)
    
    # Create a RandomOverSampler object with a random state of 42
    ros = RandomOverSampler(random_state=42)

    # Resample the 'train_pred_and_time' dataframe and 'greeks.Alpha' series using RandomOverSampler
    # The resampled data is assigned to 'train_ros' and 'y_ros' respectively
    x_ros, y_ros = ros.fit_resample(df, classi)
#     print(y_ros.value_counts())
    return x_ros, y_ros

def normolized(df):
    columns = df.columns
    
    scaler = StandardScaler()
    model = scaler.fit(df)
    scaled_df = model.transform(df)
    
    scaled_df = pd.DataFrame(scaled_df, columns = columns)
    return scaled_df

## Balanced Log Loss

In [115]:
def balanced_log_loss(y_true, y_pred):
    #number of true values of 0 & 1
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    # calculate the weights for each class to balance classes
    w_0 = 1 / N_0
    w_1 = 1 / N_1
    # calculate the predicted probabilities for each class
    p_0 = np.clip(y_pred[:, 0], 1e-15, 1 - 1e-15)
    p_1 = np.clip(y_pred[:, 1], 1e-15, 1 - 1e-15)
    # calculate the summed log loss for each class
    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0))
    log_loss_1 = -np.sum(y_true * np.log(p_1))
    # calculate the weighted summed logarithmic loss
    # (factgor of 2 included to give same result as LL with balanced input)
    balanced_log_loss = 2*(w_0 * log_loss_0 + w_1 * log_loss_1) / (w_0 + w_1)
    # return the average log loss
    return balanced_log_loss/(N_0+N_1)

In [116]:
_tmp_true = np.array([0, 0, 1, 0, 1])
_tmp_pred = np.array([
    [0, 1],
    [0, 1],
    [1, 0],
    [0, 1],
    [1, 0]
], dtype = np.float32)
_tmp_pred = 1.0 - _tmp_pred
print(_tmp_pred)


balanced_log_loss(_tmp_true, _tmp_pred)

[[1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]]


-0.0

## Model

In [117]:
tab = TabPFNClassifier(N_ensemble_configurations=12)
type(tab)
tabpfn = [TabPFNClassifier(N_ensemble_configurations=12),
          TabPFNClassifier(N_ensemble_configurations=24)]

type(tab)

Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters


tabpfn.scripts.transformer_prediction_interface.TabPFNClassifier

In [118]:
tabpfn = TabPFNClassifier(N_ensemble_configurations=12)

config1 = [
    {'name': 'xgb', 'n_estimators': 100 , 'max_depth': 3, 'learning_rate': 0.1, 'subsample': 0.9, 'colsample_bytree': 1, 'seed': 21},
    {'name': 'xgb', 'n_estimators': 150 , 'max_depth': 3, 'learning_rate': 0.2, 'subsample': 0.95, 'colsample_bytree': 0.7, 'seed': 36},
    {'name': 'xgb', 'n_estimators': 200 , 'max_depth': 4, 'learning_rate': 0.05, 'subsample': 0.85, 'colsample_bytree': 0.85, 'seed': 45},
    {'name': 'xgb', 'n_estimators': 150 , 'max_depth': 4, 'learning_rate': 0.15, 'subsample': 0.7, 'colsample_bytree': 0.85, 'seed': 65},
    {'name': 'xgb', 'n_estimators': 250 , 'max_depth': 2, 'learning_rate': 0.05, 'subsample': 0.8, 'colsample_bytree': 0.9, 'seed': 26},
    {'name': 'xgb', 'n_estimators': 100 , 'max_depth': 3, 'learning_rate': 0.1, 'subsample': 0.75, 'colsample_bytree': 1, 'seed': 11},
    {'name': 'xgb', 'n_estimators': 250 , 'max_depth': 3, 'learning_rate': 0.2, 'subsample': 0.65, 'colsample_bytree': 0.7, 'seed': 12},
    {'name': 'xgb', 'n_estimators': 300 , 'max_depth': 5, 'learning_rate': 0.15, 'subsample': 0.85, 'colsample_bytree': 0.95, 'seed': 13},
    {'name': 'xgb', 'n_estimators': 150 , 'max_depth': 4, 'learning_rate': 0.15, 'subsample': 0.7, 'colsample_bytree': 0.85, 'seed': 14},
    {'name': 'xgb', 'n_estimators': 250 , 'max_depth': 5, 'learning_rate': 0.05, 'subsample': 0.8, 'colsample_bytree': 0.9, 'seed': 15},
    {'name': 'tabpfn', 'N_ensemble_configurations': 12},
    {'name': 'tabpfn', 'N_ensemble_configurations': 24},
    {'name': 'tabpfn', 'N_ensemble_configurations': 12},
    {'name': 'tabpfn', 'N_ensemble_configurations': 24},
    {'name': 'tabpfn', 'N_ensemble_configurations': 24},
    {'name': 'randomforest', 'n_estimators': 200, 'max_depth': 4, 'random_state': 42},
    {'name': 'randomforest', 'n_estimators': 250, 'max_depth': 4, 'random_state': 11},
    {'name': 'randomforest', 'n_estimators': 200, 'max_depth': 5, 'random_state': 72},
    {'name': 'randomforest', 'n_estimators': 150, 'max_depth': 5, 'random_state': 71},
    {'name': 'gaussiannb'},
    {'name': 'multinomialnb'},
    {'name': 'kneighbors', 'n_neighbors': 10, 'algorithm': 'kd_tree', 'leaf_size': 30},
    {'name': 'kneighbors', 'n_neighbors': 10, 'algorithm': 'brute', 'leaf_size': 30},
    {'name': 'kneighbors', 'n_neighbors': 10, 'algorithm': 'kd_tree', 'leaf_size': 40},
    {'name': 'kneighbors', 'n_neighbors': 5, 'algorithm': 'brute', 'leaf_size': 50},
    {'name': 'gradientboosting'}
]

config2 = [
    {'name': 'xgb', 'n_estimators': 400 , 'max_depth': 6, 'learning_rate': 0.15, 'subsample': 0.9, 'colsample_bytree': 0.85, 'seed': 0},
    {'name': 'xgb', 'n_estimators': 300 , 'max_depth': 6, 'learning_rate': 0.2, 'subsample': 0.8, 'colsample_bytree': 0.85, 'seed': 3},
    {'name': 'xgb', 'n_estimators': 100 , 'max_depth': 3, 'learning_rate': 0.1, 'subsample': 0.9, 'colsample_bytree': 1, 'seed': 5},
    {'name': 'xgb', 'n_estimators': 150 , 'max_depth': 3, 'learning_rate': 0.2, 'subsample': 0.95, 'colsample_bytree': 0.7, 'seed': 96},
    {'name': 'xgb', 'n_estimators': 200 , 'max_depth': 4, 'learning_rate': 0.05, 'subsample': 0.85, 'colsample_bytree': 0.85, 'seed': 90},
    {'name': 'xgb', 'n_estimators': 150 , 'max_depth': 4, 'learning_rate': 0.15, 'subsample': 0.7, 'colsample_bytree': 0.85, 'seed': 68},
    {'name': 'xgb', 'n_estimators': 250 , 'max_depth': 2, 'learning_rate': 0.05, 'subsample': 0.8, 'colsample_bytree': 0.9, 'seed': 6},
    {'name': 'xgb', 'n_estimators': 100 , 'max_depth': 3, 'learning_rate': 0.1, 'subsample': 0.75, 'colsample_bytree': 1, 'seed': 99},
    {'name': 'xgb', 'n_estimators': 250 , 'max_depth': 3, 'learning_rate': 0.2, 'subsample': 0.65, 'colsample_bytree': 0.7, 'seed': 93},
    {'name': 'xgb', 'n_estimators': 300 , 'max_depth': 5, 'learning_rate': 0.15, 'subsample': 0.85, 'colsample_bytree': 0.95, 'seed': 53},
    {'name': 'xgb', 'n_estimators': 150 , 'max_depth': 4, 'learning_rate': 0.15, 'subsample': 0.7, 'colsample_bytree': 0.85, 'seed': 94},
    {'name': 'xgb', 'n_estimators': 250 , 'max_depth': 5, 'learning_rate': 0.05, 'subsample': 0.8, 'colsample_bytree': 0.9, 'seed': 75},
    {'name': 'tabpfn', 'N_ensemble_configurations': 12},
    {'name': 'tabpfn', 'N_ensemble_configurations': 24},
    {'name': 'tabpfn', 'N_ensemble_configurations': 12},
    {'name': 'randomforest', 'n_estimators': 100, 'max_depth': 5, 'random_state': 5},
    {'name': 'randomforest', 'n_estimators': 100, 'max_depth': 5, 'random_state': 50},
    {'name': 'gaussiannb'},
    {'name': 'multinomialnb'},
    {'name': 'kneighbors', 'n_neighbors': 20, 'algorithm': 'ball_tree', 'leaf_size': 30},
    {'name': 'gradientboosting'}
#     {'name': 'logisticregression', 'max_iter': 150, 'random_state': 53},
#     {'name': 'logisticregression', 'max_iter': 200, 'random_state': 64},
#     {'name': 'logisticregression', 'max_iter': 50, 'random_state': 23},
#     {'name': 'logisticregression', 'max_iter': 400, 'random_state': 59},
#     {'name': 'logisticregression', 'max_iter': 300, 'random_state': 2},
]

config3 = [
    {'name': 'xgb', 'n_estimators': 100 , 'max_depth': 7, 'learning_rate': 0.01, 'subsample': 0.8, 'colsample_bytree': 0.85, 'seed': 86},
    {'name': 'xgb', 'n_estimators': 500 , 'max_depth': 5, 'learning_rate': 0.1, 'subsample': 0.9, 'colsample_bytree': 1, 'seed': 43},
    {'name': 'xgb', 'n_estimators': 150 , 'max_depth': 3, 'learning_rate': 0.2, 'subsample': 0.95, 'colsample_bytree': 0.7, 'seed': 56},
    {'name': 'xgb', 'n_estimators': 200 , 'max_depth': 4, 'learning_rate': 0.05, 'subsample': 0.85, 'colsample_bytree': 0.85, 'seed': 74},
    {'name': 'xgb', 'n_estimators': 150 , 'max_depth': 3, 'learning_rate': 0.05, 'subsample': 0.7, 'colsample_bytree': 0.85, 'seed': 75},
    {'name': 'xgb', 'n_estimators': 250 , 'max_depth': 2, 'learning_rate': 0.05, 'subsample': 0.8, 'colsample_bytree': 0.9, 'seed': 89},
    {'name': 'tabpfn', 'N_ensemble_configurations': 64},
    {'name': 'tabpfn', 'N_ensemble_configurations': 12},
    {'name': 'randomforest', 'n_estimators': 200, 'max_depth': 7, 'random_state': 22},
    {'name': 'randomforest', 'n_estimators': 500, 'max_depth': 6, 'random_state': 77}
]

config = config1+config2+config3

def config_classifiers(config):
    CLASSIFIER_CLASSES = {
        'xgb': XGBClassifier,
        'tabpfn': TabPFNClassifier,
        'randomforest': RandomForestClassifier,
        'gaussiannb': GaussianNB,
        'multinomialnb': MultinomialNB,
        'kneighbors': KNeighborsClassifier,
        'gradientboosting': GradientBoostingClassifier,
        'logisticregression': LogisticRegression
    }

    thismodule = sys.modules[__name__]
    classifiers = []
    for sub_cfg in config:
    #     cls = globals()[sub_cfg['name']]
        cls = CLASSIFIER_CLASSES[sub_cfg['name']]
        kwargs = {k:v for k, v in sub_cfg.items() if k != 'name'}
        classifier = cls(**kwargs)
        classifiers.append(classifier)
    return classifiers

Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters


In [119]:
class Ensemble():
    def __init__(self):
        self.classifiers = config_classifiers(config)
        print(self.classifiers)
        
    def fit(self,X,y):
        for classifier in self.classifiers:
            print(classifier)
            if (type(classifier) == type(tabpfn)):
                classifier.fit(X, y, overwrite_warning=True)
            else :
                classifier.fit(X, y)
     
    def predict_proba(self, x):
        # N_models * N_rows * N_classes (#models * 5 * 4)
        probabilities = np.stack([classifier.predict_proba(x) for classifier in self.classifiers])
        averaged_probabilities = np.mean(probabilities, axis=0) # N_rows * N_classes
        class_0_est_instances = averaged_probabilities[:, 0].sum()  # N_rows
        others_est_instances = averaged_probabilities[:, 1:].sum()  # N_rows   
        # Weighted probabilities based on class imbalance
        new_probabilities = averaged_probabilities * np.array([[1/(class_0_est_instances if i==0 else others_est_instances) for i in range(averaged_probabilities.shape[1])]])
        ret =  new_probabilities / np.sum(new_probabilities, axis=1, keepdims=1) 
        return ret

# Post processing

In [120]:
def calibrate_prob(probs, shape, thres_1, thres_0):
    print('TYPE:', probs.shape, type(probs))
    
    #transfer to probabilitiy of 2 class: 0 & 1
    class_0_prob = probs[:, 0]
    others_prob = probs[:, 1:].sum(axis=1)
    class_0_prob = class_0_prob.reshape((shape, 1))
    others_prob = others_prob.reshape((shape, 1))
    
#     probs = np.concatenate([class_0_prob, others_prob], axis=-1)
#     ret = probs.copy()
    col_0 = class_0_prob.copy()
    col_0[class_0_prob < thres_1] = 0.0
    col_0[class_0_prob > thres_0] = 1.0
    col_1 = 1.0 - col_0
    ret = np.concatenate([col_0, col_1], axis = -1)
    print('ret', type(ret))
    return ret

## Training

In [121]:
def training():
    splits = 5   # Total number of splits for the inner cross-validation
    models = []   # List to store the trained models for each inner fold
    thres_lst = []
    loss_lst = []
    pred_sets = []
    true_sets = []

    # Loop over the splits of the inner cross-validation using tqdm for progress visualization
    for split in range(splits):
        model = Ensemble()
        print('fold', split)
        #loading train & test dataset for each fold
        save_dir = f'../datasets/kfold/fold{split}'

        # x_train & y_train
        df_train = pd.read_csv(os.path.join(save_dir, 'train.csv'))
        x_train = df_train.drop(['Class', 'Alpha'], axis=1)
        y_train = df_train.Alpha
        #labael-encoder
        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        # pre-processing
        x_train, y_train = prepair_input(x_train, y_train)

        # x_val & y_val
        df_val = pd.read_csv(os.path.join(save_dir, 'val.csv'))
        x_val = df_val.drop(['Class'], axis=1)
        y_val = df_val.Class
        print(y_val.value_counts())

        #fitting model
        model.fit(x_train, y_train)   # Fit the model on the training data
        models.append(model)   # Append the trained model to the list of models
        y_pred = model.predict_proba(x_val) # Predict probabilities for the validation set for 4 classes
        shape = y_val.size
        
        for i in range(shape):
            pred_sets.append(y_pred[i])
            true_sets.append(y_val[i])
    
    y_pred = np.array(pred_sets)
    y_val = pd.Series(true_sets)
    
    print('Models', models)
    
    return models, y_pred, y_val   # Return the list of trained models

In [122]:
models, y_pred, y_val = training()

Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
[XGBClassifier(base_score=None, booster=None, colsample_byle

/Users/hadangvu/anaconda3/lib/python3.10/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/hadangvu/anaconda3/lib/python3.10/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.7,
              enable_categorical=False, gamma=None, gpu_id=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.2, max_delta_step=None, max_depth=3,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=150, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, seed=36, subsample=0.95,
              tree_method=None, validate_parameters=None, verbosity=None)
[00:27:15] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/hadangvu/anaconda3/lib/python3.10/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/hadangvu/anaconda3/lib/python3.10/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.85,
              enable_categorical=False, gamma=None, gpu_id=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_delta_step=None, max_depth=4,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=200, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, seed=45, subsample=0.85,
              tree_method=None, validate_parameters=None, verbosity=None)
[00:27:16] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.85,
              enable_categorical=False, gamma=None, gpu_id=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_delta_step=None, max_depth=3,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=150, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, seed=75, subsample=0.7,
              tree_method=None, validate_parameters=None, verbosity=None)
[00:36:22] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/hadangvu/anaconda3/lib/python3.10/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/hadangvu/anaconda3/lib/python3.10/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.9,
              enable_categorical=False, gamma=None, gpu_id=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_delta_step=None, max_depth=2,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=250, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, seed=89, subsample=0.8,
              tree_method=None, validate_parameters=None, verbosity=None)
[00:36:22] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/hadangvu/anaconda3/lib/python3.10/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/hadangvu/anaconda3/lib/python3.10/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


TabPFNClassifier(N_ensemble_configurations=64)
TabPFNClassifier(N_ensemble_configurations=12)
RandomForestClassifier(max_depth=7, n_estimators=200, random_state=22)
RandomForestClassifier(max_depth=6, n_estimators=500, random_state=77)
Models [<__main__.Ensemble object at 0x2cd132f50>, <__main__.Ensemble object at 0x145f19750>, <__main__.Ensemble object at 0x288213e80>, <__main__.Ensemble object at 0x28ceff160>, <__main__.Ensemble object at 0x28c520a00>]


## Evaluation

In [123]:
def evaluation(y_pred, y_val):    
    ret = []
    #find best threshold
    for thres_1 in np.arange(0, 1, 0.01):
        for thres_0 in np.arange(thres_1, 1, 0.01):
            shape = len(y_val)
            #post processing
            y_p = calibrate_prob(y_pred, shape, thres_1, thres_0)

            #balanced log loss
            loss = balanced_log_loss(y_val, y_p)  # Calculate the balanced log loss between the predicted labels and the true labels

    #         # checking
    #         y_val = y_val.to_frame()
    #         y_val.rename(columns = {'Class': 'gt'}, inplace = True)
    #         y_val['pred'] = y_p[:, 1]
    # #         print(type(y_val['gt']), type(y_val.loc[0, 'gt']), type(y_val['pred']), type(y_val.loc[0, 'pred']))
    #         p00 = y_p[:, 1]
    #         p00 = p00.flatten()
    #         y_val['prob'] = p00
    #         display(y_val)
            print('>LOSS=%.5f' % loss)
            ret.append([thres_1, thres_0, loss])
    
    ret = sorted(ret, key= lambda x: x[2])
    print('best:\n', ret[:10])
    
    return ret[0]

In [124]:
best_thres = evaluation(y_pred, y_val)

TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=9.97489
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=9.69788
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=9.32862
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=8.31337
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=7.75974
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=6.74519
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=6.09974
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=5.54826
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=5.08890
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=4.63106
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=4.54027
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=4.07996
TYPE: (617, 4) <class 'numpy

>LOSS=0.21136
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=0.21356
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=0.21488
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=0.21635
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=0.21739
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=0.21782
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=0.21875
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=0.21999
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=0.22061
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=0.22126
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=0.22179
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=0.22228
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=0.22260
TYPE: (617, 4)

ret <class 'numpy.ndarray'>
>LOSS=1.84999
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.85060
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.66872
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.57753
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.57753
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.57753
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.57916
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.57968
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.48972
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.49021
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.49119
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.49166
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>


>LOSS=1.54127
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.54377
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.36714
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.28075
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.28418
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.28968
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.29183
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.29594
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.30068
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.21594
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.13066
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.04400
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.04894
TYPE: (617, 4)

ret <class 'numpy.ndarray'>
>LOSS=1.98524
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.98636
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.98710
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.98746
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.98883
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.99084
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.99181
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.99431
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.81768
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.73129
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.73472
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=1.74023
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>


ret <class 'numpy.ndarray'>
>LOSS=7.04951
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=7.04983
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=7.05029
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=7.05065
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=7.05083
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=7.05090
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=7.05090
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=7.05090
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=7.05090
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=7.05090
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=7.05090
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
>LOSS=7.21171
TYPE: (617, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>


## Infer

In [137]:
#pre-processing
Id, test = prepair_test(test)



In [139]:
num_ensembles = len(models)
y_p_lst = list()

for ensemble in models:
    y_pred = ensemble.predict_proba(test)
    shape = test.shape[0]
    y_p = calibrate_prob(y_pred, shape, best_thres[0], best_thres[1])
    y_p_lst.append(y_p)

y_p = np.array(y_p_lst).sum(axis=0)/num_ensembles

TYPE: (20, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
TYPE: (20, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
TYPE: (20, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
TYPE: (20, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>
TYPE: (20, 4) <class 'numpy.ndarray'>
ret <class 'numpy.ndarray'>


In [140]:
#post processing
shape = test.shape[0]

submission = pd.DataFrame(Id, columns=['Id'])
submission["class_0"] = y_p[:, 0]
submission["class_1"] = y_p[:, 1]
submission.to_csv('submission.csv', index=False)

In [144]:
loss = balanced_log_loss(a.Class, y_p)
loss

0.07226396249497796

In [141]:
df_submission = pd.read_csv('submission.csv')
df_submission

,Id,class_0,class_1
0,9b9e6e21b830,0.766163,0.233837
1,b358f0fdb95b,0.583543,0.416457
2,ffcca4ded3bb,1.000000,0.000000
3,5d694ba9aa16,0.014357,0.985643
4,ba0c42e12f1f,0.733145,0.266855
5,4c92a971cf91,0.699596,0.300404
6,da1a5aec1818,0.049112,0.950888
7,1a9fecf65695,0.829510,0.170490
8,332a3850302f,1.000000,0.000000
9,2cddd2d70d72,0.000000,1.000000


In [130]:
a = pd.read_csv('../datasets/icr-identify-age-related-conditions/train.csv')

In [131]:
a = a.sample(20)

In [134]:
test = a.drop(['Class'], axis=1)

In [136]:
test.shape

(20, 57)

In [102]:
Id , test = prepair_test(a)

/Users/hadangvu/anaconda3/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/Users/hadangvu/anaconda3/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/Users/hadangvu/anaconda3/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated

In [103]:
best_thres[2]

0.176437112832583

In [104]:
y_val = test.Class
test = test.drop(['Class'], axis=1)

In [107]:
ensemble = models[0]

y_pred = ensemble.predict_proba(test)
print(y_pred)
shape = test.shape[0]
y_p = calibrate_prob(y_pred, shape, best_thres[0], best_thres[1])
loss = balanced_log_loss(y_val, y_p)



print(y_p)
print(y_val)
print(loss)

[[0.0150184  0.97660683 0.00475821 0.00361655]
 [0.91013772 0.03044561 0.04216328 0.01725339]
 [0.23939487 0.74559242 0.00799461 0.0070181 ]
 [0.71980441 0.10288747 0.08039475 0.09691337]
 [0.66116159 0.05957084 0.06742329 0.21184428]
 [0.04558401 0.93268208 0.0125292  0.00920471]
 [0.92986021 0.0236091  0.03116548 0.01536521]
 [0.61989514 0.27334312 0.06626983 0.04049191]
 [0.70154113 0.04703769 0.12069711 0.13072407]
 [0.67498496 0.28656156 0.0244525  0.01400098]
 [0.67974516 0.11320133 0.02249874 0.18455477]
 [0.60997036 0.12503957 0.02607852 0.23891156]
 [0.54718602 0.2776101  0.00920867 0.16599521]
 [0.01136154 0.92890694 0.05503061 0.00470092]
 [0.74143032 0.06579484 0.10255189 0.09022295]
 [0.42182102 0.49280779 0.07745074 0.00792046]
 [0.67643141 0.11520324 0.04150181 0.16686354]
 [0.61999488 0.316123   0.03056494 0.03331717]
 [0.60484306 0.19577769 0.11170487 0.08767438]
 [0.82758634 0.14467183 0.01418133 0.0135605 ]
 [0.90231939 0.03535361 0.03082581 0.0315012 ]
 [0.85446614 

In [ ]:
loss_ensembles = list()
for ensemble in models:
    y_pred = ensemble.predict_proba(test)
    print(y_pred)
    shape = test.shape[0]
    y_p = calibrate_prob(y_pred, shape, best_thres[0], best_thres[1])
    loss = balanced_log_loss(y_val, y_p)
    loss_ensembles.append(loss)
    final_loss = np.mean(loss_ensembles)

print('lost_ensembles', loss_ensembles)
print('LOSS', final_loss)

In [ ]:
y_pred.shape

In [ ]:
shape = test.shape[0]

y_p = calibrate_prob(y_pred, shape, 0.90)
y_p.shape

In [ ]:
y_val = a.Class
loss = balanced_log_loss(y_val, y_p)
loss

In [ ]:


submission = pd.DataFrame(Id, columns=['Id'])
submission["class_0"] = y_p[:, 0]
submission["class_1"] = y_p[:, 1]
submission.to_csv('submission.csv', index=False)

In [ ]:
submission